In [29]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup


# Konfigurasi Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s, options=options)

# Inisialisasi list untuk menyimpan data
data_list = []

# Fungsi untuk scraping detail kunci
def scrape_key_details(soup):
    car_data = {
        'Kondisi': "N/A",
        'Tahun Kendaraan': "N/A",
        'Kilometer': "N/A",
        'Warna': "N/A",
        'Cakupan mesin': "N/A",
        'Transmisi': "N/A",
        'Penumpang': "N/A"
    }
    specs = soup.select(".c-key-details__item")
    for spec in specs:
        try:
            key = spec.select_one("span.u-text-7").text.strip()
            value = spec.select_one("span.u-text-bold").text.strip()
            if key in car_data:
                car_data[key] = value
        except Exception as e:
            print(f"Error parsing key detail: {e}")
            continue
    return car_data

# Fungsi untuk scraping spesifikasi tambahan
def scrape_specifications(soup):
    additional_data = {
        'Pintu': "N/A",
        'Dirakit': "N/A",
        'Tipe Bahan Bakar': "N/A"
    }
    spec_tab = soup.select_one("#tab-specifications")
    if spec_tab:
        spec_items = spec_tab.select(".u-border-bottom.u-padding-ends-xs.u-flex.u-flex--justify-between")
        for item in spec_items:
            try:
                key = item.select("span")[0].text.strip()
                value = item.select("span")[1].text.strip()
                if key in additional_data:
                    additional_data[key] = value
            except Exception as e:
                print(f"Error parsing specification: {e}")
                continue
    return additional_data

# Fungsi untuk scraping satu halaman
def scrape_page(page_number):
    url = f'https://www.mobil123.com/mobil-dijual/indonesia?type=used&page_number={page_number}&page_size=25'
    driver.get(url)
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h2.listing__title a"))
        )
    except TimeoutException:
        print(f"TimeoutException at page {page_number}")
        return

    car_elements = driver.find_elements(By.CSS_SELECTOR, "h2.listing__title a")
    car_urls = [car.get_attribute('href') for car in car_elements]

    for car_url in car_urls:
        driver.get(car_url)
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h1.u-text-bold"))
            )
        except TimeoutException:
            print(f"TimeoutException at car URL: {car_url}")
            continue

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        car_details = {}

        try:
            car_details['Title'] = soup.select_one("h1.u-text-bold").text.strip()
        except Exception as e:
            print(f"Error parsing title: {e}")
            car_details['Title'] = "N/A"

        try:
            car_details['Harga'] = soup.select_one("div.listing__price.u-text-4.u-text-bold").text.strip()
        except Exception as e:
            print(f"Error parsing price: {e}")
            car_details['Harga'] = "N/A"

        # Scraping key details
        key_details = scrape_key_details(soup)
        car_details.update(key_details)

        # Scraping additional specifications
        specifications = scrape_specifications(soup)
        car_details.update(specifications)

        data_list.append(car_details)
        time.sleep(1)  # Tambahkan jeda 1 detik antara permintaan detail setiap mobil

# Scraping dua halaman pertama sebagai contoh
for page in range(1, 10):
    scrape_page(page)
    time.sleep(5)  # Tambahkan jeda 5 detik antara permintaan setiap halaman

# Simpan ke CSV
if data_list:
    keys = data_list[0].keys()
    with open('car_details.csv', 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Tutup driver
driver.quit()

Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute 'text'
Error parsing price: 'NoneType' object has no attribute

In [62]:
import pandas as pd
df = pd.read_csv('car_details.csv')
df

,Title,Harga,Kondisi,Tahun Kendaraan,Kilometer,Warna,Cakupan mesin,Transmisi,Penumpang,Pintu,Dirakit,Tipe Bahan Bakar
0,2013 Toyota Camry V Sedan 2.5,Rp 188.000.000,Mobil Bekas,2013,105 - 110K km,Hitam,2494 cc,Automatic,5,4,NaN,Pertamax
1,2014 Ford EcoSport 1.5 Titanium SUV Automatic ...,NaN,Mobil Bekas,2014,70 - 75K km,Abu-abu,1499 cc,Automatic,5,5,Complete Built Up (CBU),Pertamax
2,"2022 Mercedes-Benz GLC200 2,0 AMG Line GLC 200...",Rp 920.000.000,Mobil Bekas,2022,21000 km,Hitam,1991 cc,Automatic,5,5,Complete Knock Down (CKD),Pertamax
3,2019 Suzuki SX4 S-Cross 1.5 Hatchback,Rp 209.000.000,Mobil Bekas,2019,45 - 50K km,Putih,1491 cc,Automatic,5,5,Complete Built Up (CBU),Pertamax
4,2021 Toyota Agya 1.2 TRD Hatchback,NaN,Mobil Bekas,2021,25 - 30K km,Merah,1197 cc,Automatic,5,5,Complete Knock Down (CKD),Pertamax
...,...,...,...,...,...,...,...,...,...,...,...,...
220,"KM 49RB!! 2015 Toyota Agya 1,0 G Hatchback",Rp 95.000.000,Mobil Bekas,2015,49000 km,Putih,998 cc,Automatic,5,5,NaN,Pertamax
221,2022 Mercedes-Benz CLA200 1.3 AMG Line Coupe N...,Rp 845.000.000,Mobil Bekas,2022,10000 km,Putih,1332 cc,Automatic,5,4,Complete Built Up (CBU),Pertamax
222,2022 Mercedes-Benz C200 1.5 Avantgarde Line Se...,Rp 845.000.000,Mobil Bekas,2022,9000 km,Putih,1496 cc,Automatic,5,4,Complete Knock Down (CKD),Pertamax
223,2019 Mercedes-Benz C300 2.0 AMG Sedan Nik2019 ...,Rp 695.000.000,Mobil Bekas,2019,16000 km,Hitam,1991 cc,Automatic,5,4,Complete Knock Down (CKD),Pertamax


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Title             225 non-null    object
 1   Harga             191 non-null    object
 2   Kondisi           225 non-null    object
 3   Tahun Kendaraan   225 non-null    int64 
 4   Kilometer         225 non-null    object
 5   Warna             225 non-null    object
 6   Cakupan mesin     225 non-null    object
 7   Transmisi         225 non-null    object
 8   Penumpang         225 non-null    int64 
 9   Pintu             225 non-null    int64 
 10  Dirakit           212 non-null    object
 11  Tipe Bahan Bakar  225 non-null    object
dtypes: int64(3), object(9)
memory usage: 21.2+ KB


In [64]:
from preprocessing import preprocess_data

In [65]:
# Memanggil fungsi preprocessing
df_preprocessed = preprocess_data(df)
df_preprocessed

d:\MURYOKUSHO\Kuliah\Semester 6\Machine Learning\Project UAS\preprocessing.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['brand'] = df['Title'].apply(lambda x: extract_brand_name(x, brand_list))
d:\MURYOKUSHO\Kuliah\Semester 6\Machine Learning\Project UAS\preprocessing.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
d:\MURYOKUSHO\Kuliah\Semester 6\Machine Learning\Project UAS\preprocessing.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

,brand,jenis_mobil,Tahun Kendaraan,Warna,Transmisi,Kilometer,Cakupan mesin,Tipe Bahan Bakar,Dirakit,Penumpang,Pintu,Harga
3,Suzuki,Hatchback,2019,Putih,Automatic,47500,1491,Pertamax,Complete Built Up (CBU),5,5,209.000.000
8,Lexus,SUV,2016,Putih,Automatic,67500,1998,Pertamax,Complete Built Up (CBU),5,5,550.000.000
11,Honda,Sedan,2012,Hitam,Automatic,52500,2354,Pertamax,Complete Built Up (CBU),5,4,155.000.000
12,BMW,Sedan,2016,Silver,Automatic,87500,1998,Pertamax,Complete Knock Down (CKD),5,4,405.000.000
16,Nissan,MPV,2013,Hitam,Automatic,117500,3498,Pertamax,Complete Built Up (CBU),7,5,300.000.000
...,...,...,...,...,...,...,...,...,...,...,...,...
218,Toyota,Wagon,2022,Putih,Automatic,15000,2494,Pertamax,Complete Built Up (CBU),7,5,1.150.000.000
219,Hyundai,MPV,2013,Abu-abu,Automatic,107500,2359,Pertamax,Complete Knock Down (CKD),9,5,159.000.000
221,Mercedes-Benz,Coupe,2022,Putih,Automatic,10000,1332,Pertamax,Complete Built Up (CBU),5,4,845.000.000
222,Mercedes-Benz,Sedan,2022,Putih,Automatic,9000,1496,Pertamax,Complete Knock Down (CKD),5,4,845.000.000


In [66]:
df_preprocessed['brand'].unique()

array(['Suzuki', 'Lexus', 'Honda', 'BMW', 'Nissan', 'Toyota', 'Ferrari',
       'Mitsubishi', 'Daihatsu', 'Hyundai', 'Mercedes-Benz', 'Isuzu',
       'Mazda', 'Wuling', 'Chevrolet', 'Ford', 'Volkswagen'], dtype=object)

In [67]:
columns_list = df_preprocessed.columns.tolist()
print(columns_list)

['brand', 'jenis_mobil', 'Tahun Kendaraan', 'Warna', 'Transmisi', 'Kilometer', 'Cakupan mesin', 'Tipe Bahan Bakar', 'Dirakit', 'Penumpang', 'Pintu', 'Harga']


In [68]:
df_preprocessed

,brand,jenis_mobil,Tahun Kendaraan,Warna,Transmisi,Kilometer,Cakupan mesin,Tipe Bahan Bakar,Dirakit,Penumpang,Pintu,Harga
3,Suzuki,Hatchback,2019,Putih,Automatic,47500,1491,Pertamax,Complete Built Up (CBU),5,5,209.000.000
8,Lexus,SUV,2016,Putih,Automatic,67500,1998,Pertamax,Complete Built Up (CBU),5,5,550.000.000
11,Honda,Sedan,2012,Hitam,Automatic,52500,2354,Pertamax,Complete Built Up (CBU),5,4,155.000.000
12,BMW,Sedan,2016,Silver,Automatic,87500,1998,Pertamax,Complete Knock Down (CKD),5,4,405.000.000
16,Nissan,MPV,2013,Hitam,Automatic,117500,3498,Pertamax,Complete Built Up (CBU),7,5,300.000.000
...,...,...,...,...,...,...,...,...,...,...,...,...
218,Toyota,Wagon,2022,Putih,Automatic,15000,2494,Pertamax,Complete Built Up (CBU),7,5,1.150.000.000
219,Hyundai,MPV,2013,Abu-abu,Automatic,107500,2359,Pertamax,Complete Knock Down (CKD),9,5,159.000.000
221,Mercedes-Benz,Coupe,2022,Putih,Automatic,10000,1332,Pertamax,Complete Built Up (CBU),5,4,845.000.000
222,Mercedes-Benz,Sedan,2022,Putih,Automatic,9000,1496,Pertamax,Complete Knock Down (CKD),5,4,845.000.000
